In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import datetime
import os.path
import sys

import pandas as pd
import numpy as np

import backtrader as bt

# Strategy

In [ ]:
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('live', False),
        ('walkforward', None),
    )
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        # To keep track of pending orders and buy price/commission

        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Trim the indicators if we are running live, just need the current one
        if self.params.live:
            pass
        
        self.d = {
            'sma': bt.indicators.SimpleMovingAverage( self.datas[0],  period= self.params.maperiod),
        }
        
        if self.params.walkforward:
            self.wfsma = []
            for run in self.params.walkforward:
                params=dict(run['best_insample_param'])
                self.wfsma.append({
                    'sma': bt.indicators.SimpleMovingAverage(self.datas[0], period=params['maperiod']),
                    'test_period': run['oos'],
                    'train_param': params,
                }) 
        
    def set_wf_params(self):
        for row in self.wfsma:
            start, end = row['test_period']
            if self.datetime.datetime() >= start and self.datetime.datetime() < end:
                self.d = row
            
    def next(self):
        if self.params.walkforward: self.set_wf_params()
        if self.order: return
        
        if not self.position:
            if  self.dataclose[0] > self.d['sma'][0]:  
                self.order = self.buy()
        else:
            if  self.dataclose[0] < self.d['sma'][0]:
                self.order = self.sell()

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            self.bar_executed = len(self)
        # Write down: no pending order
        self.order= None



## Analyzers

In [ ]:
class myAnalyzer(bt.Analyzer):
    params=(
        ('printlog', None),
    )
    
    def __init__(self):
        self.equitydf=pd.DataFrame()

    def log(self, txt, dt=None, doprint=False):
        if self.p.printlog or doprint:
            dt= dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')
            
    def start(self):
        self.trade_count = 0
        self.starting_nav= self.strategy.broker.get_value() 
        
        self.num_trades = 0
        self.first_trade_open = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            if order.isbuy():
                self.log(f'BUY CREATE, {self.strategy.dataclose[0]:.2f}')
            elif order.issell():
                self.log(f"SELL CREATE {self.strategy.dataclose[0]:.2f}")
                
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

    def notify_trade(self, trade):
        if not self.first_trade_open:
            self.first_trade_open = self.strategy.datetime.datetime()
            self.equitydf.at[self.first_trade_open, 'equity'] = self.starting_nav

        if trade.isclosed:
            self.num_trades += 1
            self.equitydf.at[self.strategy.datetime.datetime(), 'equity'] = self.strategy.broker.getvalue()
            
        self.trade_count += 1 # below
        if not trade.isclosed:
            return
        self.log(f'OPERATION PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}')
    
    def stop(self):
        self.ending_nav = self.strategy.broker.getvalue()
        self.params= self.strategy.params
        self.log(f'''{vars(self.params)} \tEnding Value {self.ending_nav:.2f}''', doprint=True)

    def get_analysis(self):
        
        self.equitydf['net_profit'] = self.equitydf['equity'].diff()

        ret= {  'equitydf':self.equitydf,
                'trade_count': self.trade_count,
                'ending_NAV': self.ending_nav,
                'P&L' : self.ending_nav-self.starting_nav }
        ret.update(**vars(self.params))
        return ret

## Backtest set-up

In [ ]:
def get_data(fname_symbol = 'CL', folder_name = '5min', suffix = '5min_20160103_20190405'):
    df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
    df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
    df.columns = [col_name.lower() for col_name in df.columns]
    df = df.dropna()
    return fname_symbol,df

def set_broker_parameters(cerebro):
    cerebro.broker.setcash(100000.)
    cerebro.broker.setcommission(commission=2.5, margin=6000.0, mult=1000.0, name='CL')
    cerebro.broker.set_slippage_fixed(.01, 
                              slip_open=True, 
                              slip_limit=True, 
                              slip_match=False, 
                              slip_out=True)

def set_analyzers(cerebro):    
    cerebro.addanalyzer(myAnalyzer, printlog=None)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer)
    cerebro.addanalyzer(bt.analyzers.Transactions)
    cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Days, _name='dayreturn')
    cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, riskfreerate=0,_name='mysharpe',timeframe=bt.TimeFrame.Days)#, compression=30)

def set_opt_analyzers(cerebro):
    cerebro.addanalyzer(myAnalyzer, printlog=None)
    cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Days, _name='dayreturn')

def set_sizers(cerebro):
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)
    
def get_periods(start_is, is_duration, oos_duration, wf_offset,wf_cycles):
    periods=[]
    start_is=pd.Timestamp(start_is)
    start_oos=start_is+pd.DateOffset(**is_duration)
    end_oos=start_oos+pd.DateOffset(**oos_duration)
    for i in range(wf_cycles):
        periods.append({'run':i,
                        'is':(start_is, start_oos),
                        'oos':(start_oos, end_oos)})
        start_is+=pd.DateOffset(**wf_offset)
        start_oos+=pd.DateOffset(**wf_offset)
        end_oos+=pd.DateOffset(**wf_offset)
    return periods
    
periods= get_periods(
    start_is = "2017-01-01", 
    is_duration={'months':3}, 
    oos_duration={'months':1},
    wf_offset={'months':1},
    wf_cycles=14
)

periods

## Optimization

In [ ]:
cerebro = bt.Cerebro()

start_date = periods[0]['is'][0]
end_date = periods[-1]['is'][1]
print('Start/End: {} - {}'.format(start_date, end_date))

# Add Strategy
opt_args=dict(
    maperiod=range(10, 31, 5))

cerebro.optstrategy(TestStrategy, **opt_args)

set_broker_parameters(cerebro)

set_opt_analyzers(cerebro)

set_sizers(cerebro)

data_name,df=get_data()

cerebro.adddata(bt.feeds.PandasData(dataname = df[start_date:end_date]),name=data_name)

results=cerebro.run(maxcpu=5)

## Select per-period insample winners

In [ ]:
# Get in-sample and out-of-sample best parameters with correct pre-allocation
for period in periods:
    # Get the insample time period
    insample_start = period['is'][0]
    insample_end = period['is'][1]
    print('In-Sample Period: {} - {}'.format(insample_start, insample_end))
    
    # Put the best parameters and best profit into a list
    period_netprofit = []
    for [result] in results:
        myAnalysis=result.analyzers.myanalyzer.get_analysis()
        df = myAnalysis['equitydf']
        optim=[(key, myAnalysis[key]) for key in opt_args]
        
        period_profit = df[insample_start:insample_end]['net_profit'].sum()
        print(f'\t{dict(optim)}: {period_profit}')
        period_netprofit.append((optim, period_profit,df))

        
    # Get the best profit parameters
    max_params, max_profit, best_equity = max(period_netprofit, key=lambda x:x[1])
    
    # Add it to the dictionary
    print(f'Best is: {dict(max_params)} with {max_profit} profit\n')
    period['best_insample_param'] = max_params
    period['best_insample_profit'] = max_profit
    period['best_equity'] = best_equity

## Walk-forward Efficiency

In [ ]:
# Get in-sample and out-of-sample best parameters with correct pre-allocation
all_walkforward_efficiency = []
all_is_profits = []
all_oos_profits=[]
for period in periods:
    # Get the insample time period
    oos_start, oos_end = period['oos']
    print(f'Out-of-Sample Period: {oos_start} - {oos_end}')

    insample_profit = period['best_insample_profit']
    oos_profit = period['best_equity']['net_profit'][oos_start:oos_end].sum()
    
    insample_annual_profit = 365/ (period['is'][1]-period['is'][0]).days * insample_profit
    oos_annual_profit = 365/(period['oos'][1]-period['oos'][0]).days * oos_profit
 
    # Calculate WFE
    sgn = 1 if insample_profit > 0 else -1
    period['wfe'] = sgn*oos_annual_profit/ insample_annual_profit * 100 
    

    all_walkforward_efficiency.append(period['wfe'])
    all_is_profits.append(insample_annual_profit)
    all_oos_profits.append(oos_annual_profit)
    
    print(f'\t Period Profit IS:{insample_profit:0.2f} / OOS:{oos_profit:0.2f}')
    print(f'\t Annualized Profit IS:{insample_annual_profit:0.2f} / OOS:{oos_annual_profit:0.2f}')
    print(f"\t WFE: {period['wfe']:0.1f}%")

wfe = sum(all_walkforward_efficiency) / len(all_walkforward_efficiency) 


print(f'Average WFE: {wfe:0.1f}%')
    

wfe2 = 100*sum(all_oos_profits) / sum(all_is_profits) 

print(f'Effective WFE[2]: {wfe2:0.1f}%')

## Walk-forward Equity Curve

In [ ]:
cerebro = bt.Cerebro()

start_date = periods[0]['oos'][0]
end_date = periods[-1]['oos'][1]
print('Start/End: {} - {}'.format(start_date, end_date))

cerebro.addstrategy(TestStrategy, walkforward=periods)

set_broker_parameters(cerebro)

set_analyzers(cerebro)

set_sizers(cerebro)

data_name,df=get_data()

cerebro.adddata(bt.feeds.PandasData(dataname = df[start_date:end_date]),name=data_name)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

[result]=cerebro.run()

In [ ]:
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
dr = result.analyzers.dayreturn.get_analysis()
df = pd.DataFrame(dr.items()).set_index(0)#.loc[:,1]
cumsum= df.cumsum()
r = result.analyzers.myanalyzer.get_analysis()
plt.figure(figsize=(10,5))
plt.plot(r['equitydf']['net_profit'].cumsum())##cumsum)
plt.plot(cumsum*(r['ending_NAV']-r['P&L']))
plt.title('Walkforward Equity Curve')
plt.xlabel('Trades')
plt.ylabel('Equity')

In [ ]:
trades=result.analyzers.myanalyzer.get_analysis()['equitydf']['net_profit'].iloc[1:]
trades.to_pickle("20190717-fa4had-WORKFLOW-05.01-walkforward_submission.pkl")